# [1.3: Real Data] — Applying kNN and Evaluating on Realistic Data

Until now, you have worked with a small toy dataset to learn how kNN functions and how to evaluate recommender systems.
Now we will see how kNN performs on **more realistic data**.

The dataset we use here comes from **MovieLens**, a movie recommendation platform run by the GroupLens research lab at the University of Minnesota. MovieLens datasets are widely used in research and education because they contain real user ratings and are carefully curated for experimentation.

However, we are **not** using the full raw dataset. Collaborative filtering methods such as kNN are heavily influenced by **long-tail effects**: most movies receive very few ratings, and most users rate only a small number of movies. If we applied kNN directly to the full MovieLens dataset, it would perform very poorly (the algorithm simply would not have enough overlapping rating information to make reliable predictions).

To reduce this issue, we limit ourselves to the **most active users** and the **most frequently rated movies**.
The file `ratings1_18M.csv` includes only:

* movies with **1000+ ratings**, and
* users with **400+ ratings**.

This gives us a much denser dataset where kNN can operate effectively.

This filtering step may feel a bit **contrived**, but it is quite realistic: In industry settings, multiple algorithms are often used together, each specializing in different parts of the data. It is easy to imagine a hybrid system in which kNN is responsible only for the most common cases (popular items and highly active users), while other models handle the long tail.

## Getting Started

### Libraries

Let's start by loading the libraries we'll need by running the cell below:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pooch

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, precision_score, recall_score

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

### Download data

Now download and load the required data and helper files:

* **`ratings1_18M.csv`**: the full filtered dataset (popular movies and active users)
* **`ratings1_18M_train.csv`** and **`ratings1_18M_test.csv`**: an 80/20 train–test split of the same data

These files will allow us to run kNN on a realistic dataset and evaluate its performance properly.

In [ ]:
DATA_REPO = "https://raw.githubusercontent.com/uvapl/recommender-systems/main/data/m1/"

print("downloading data files")
for fname in ["ratings1_18M.csv", "ratings1_18M_train.csv", "ratings1_18M_test.csv"]:
    pooch.retrieve(url = DATA_REPO + fname, known_hash=None, fname=fname, path="data", progressbar=True)
for fname in ["helpers_m1.py", "pr_curve.png", "tests_m1.py"]:
    pooch.retrieve(url = DATA_REPO + fname, known_hash=None, fname=fname, path=".", progressbar=True)
print("done")

import helpers_m1
import tests_m1

ratings_df = pd.read_csv("data/ratings1_18M.csv")[["userId", "movieId", "rating"]]
ratings_df_train = pd.read_csv("data/ratings1_18M_train.csv")[["userId", "movieId", "rating"]]
ratings_df_test = pd.read_csv("data/ratings1_18M_test.csv")[["userId", "movieId", "rating"]]
display(ratings_df)

# kNN (again)

In this section, we shift to a more realistic prediction task. Previously, we predicted **multiple users’ ratings for a single movie**. Now we will predict ratings for **many different (user, movie)** combinations. This gives us a much clearer and more realistic picture of how kNN performs in practice.

Why didn’t we do this before? This approach does *not* work smoothly with scikit-learn. A robust kNN regression implementation that handles all edge cases would take a lot of time. As this would not help your conceptual understanding, we decided to just provide you with the necessary implementation in the `helper.py` file.

## Provided functions

The file `helper.py` includes two key functions:

* `build_utility_matrices()`
  This computes:

  * the utility matrix (pivot table with users as rows, movies as columns)
  * its mean-centered version
  * the average rating for each user

* `predict_user_based_knn()`
  This runs a full user-based kNN prediction procedure for all *(user, movie)* pairs in the test set.

  * It returns a DataFrame containing predictions such as:

  <table border="1" class="dataframe">
  <thead><tr style="text-align: right;"><th></th><th>userId</th><th>movieId</th><th>pred_rating</th></tr></thead>
  <tbody>
  <tr><th>0</th><td>198</td><td>19</td><td>2.256578</td></tr>
  <tr><th>1</th><td>198</td><td>158</td><td>2.313034</td></tr>
  <tr><th>2</th><td>198</td><td>165</td><td>3.498298</td></tr>
  <tr><th>3</th><td>198</td><td>173</td><td>2.302315</td></tr>
  <tr><th>…</th><td>…</td><td>…</td><td>…</td></tr>
  </tbody>
  </table>
  
  * This output can be compared directly to the true ratings in the test set.
  * Note that the predicted ratings are the *actual ratings* (scale 1 - 5) and not mean-centered ratings.

### Some details on the implementation

You do **not** need to fully understand how the function is implemented. However, it is important to be aware that real-world data introduces many edge cases that simple kNN logic cannot handle gracefully.

The prediction function is designed to ensure that:

* No prediction attempt fails due to missing data.
* Predictions are never based on zero or meaningless information.
* Cold-start users, cold-start items, sparse neighborhoods, and degenerate similarity values are all handled safely.
* The final output cleanly separates predictions from ground truth data.

To achieve this, the function includes solutions for several common edge cases:

1. **Cold-start users**
   Users in the test set who have **no training ratings** cannot be compared to anyone.
   Solution: Use the **global average rating**.

2. **Cold-start items**
   Movies in the test set that have **no ratings in training** cannot receive a neighborhood-based estimate.
   Solution: Use the **user’s average rating**.

3. **Neighbors who never rated the target movie**
   Solution: These neighbors are excluded rather than treated as zeros.

4. **Too few usable neighbors**
   If hardly any neighbors have rated the target movie, similarity-based prediction becomes unreliable.
   Solution: Fall back to the **user’s average rating**.

5. **Zero similarity information**
   If similarities sum to zero (e.g., all zeros after filtering), the model cannot get a meaningful weighted average.
   Solution: Again revert to the **user’s average**.

6. **Self-similarity**
   kNN search often returns the user as their own closest neighbor.
   Solution: The model explicitly removes this to avoid trivial predictions.

These cases are all needed to allow kNN to work robustly on large, (potentially) sparse, real-world datasets.

## Run

Now run the cell below to apply kNN to the full dataset and compute the baseline.
Note that this may take a few minutes.

In [ ]:
utility_train, utility_centered, user_means = helpers_m1.build_utility_matrices(ratings_df_train)

ratings_df_pred = helpers_m1.predict_user_based_knn(utility_train, utility_centered, user_means, ratings_df_test, k = 20)
ratings_df_baseline_user_mean = helpers_m1.predict_user_mean(utility_train, utility_centered, user_means, ratings_df_test)

display(ratings_df_pred.head())
display(ratings_df_baseline_user_mean.head())
display(ratings_df_test.head())

# MSE

We now have predicted ratings from kNN as well as baseline predictions. To evaluate how well they match the true ratings in the test set, we can compute the **Mean Squared Error (MSE)**.

Instead of using our own implementation, we will now use the standard `mean_squared_error` function from scikit-learn.
Run the cell below to compute and compare the MSE values.

In [ ]:
mse_predicted = mean_squared_error(ratings_df_test["rating"], ratings_df_pred["pred_rating"])
mse_baseline = mean_squared_error(ratings_df_test["rating"], ratings_df_baseline_user_mean["pred_rating"])
print(f"MSE predicted = {mse_predicted:.3f}")
print(f"MSE baseline = {mse_baseline:.3f}")

# Precision and Recall

To compute precision and recall, we again need to treat the problem as a **classification task**. That means we must know, for each (user, movie) pair in the **test set**, whether the user actually *liked* the movie.

Unfortunately, our dataset does not contain an explicit “liked / not liked” label.
So we will approximate it (a common practice in recommender-system evaluation). We use the **actual rating** as a proxy:

* If a rating is **greater than 3.5**, we assume the user liked the movie (`True`).
* Otherwise, we assume they did not (`False`).

Run the cell below to create a `liked_test` Series with `True`/`False` values indicating whether each test-case movie was actually liked.

In [ ]:
T = 3.5
liked_test = ratings_df_test["rating"] >= T

Once we have that information, we need to convert our **predicted ratings** into **recommendation decisions**. This works the same way as before: we apply a **threshold** to determine whether a predicted rating is high enough to justify recommending the movie.

This gives us a set of predicted labels (`True`/`False`) that we can directly compare to the actual `liked_test` labels when computing precision and recall.

Run the cell below to generate these recommendation labels for both the kNN model and the baseline.

In [ ]:
t_pred = 3.9

recommendations_knn = ratings_df_pred["pred_rating"] >= t_pred
recommendations_mean = ratings_df_baseline_user_mean["pred_rating"] >= t_pred

### Question 1

*2 pts.*

Use the `precision_score()` and `recall_score()` functions from scikit-learn (already imported above) to compute the **precision** and **recall** for both the kNN model and the baseline.
Use the previously computed variables:

* `liked_test`
* `recommendations_knn`
* `recommendations_mean`

Print the results.

In [ ]:
# your code here

print(f"          |  knn | mean ")
print(f"precision | {precision_knn:.2f} | {precision_mean:.2f}")
print(f"recall    | {recall_knn:.2f} | {recall_mean:.2f}")

In [ ]:
# test your solution
tests_m1.real_data_01(precision_knn, precision_mean, recall_knn, recall_mean)

# F1 Score

As mentioned in the previous notebook, **precision** and **recall** often behave in opposing ways. Improving one can worsen the other. For example, increasing the recommendation threshold may raise **precision** (because only highly confident recommendations are shown), but this usually lowers **recall**, since fewer potentially relevant items are included.
  
Because these two metrics can move in different directions, we often want a **single** score that captures both aspects of performance. A common way to do that is with the **F1 score**, defined as:

$
F_1 = 2 \cdot \frac{\text{precision} \cdot \text{recall}}
{\text{precision} + \text{recall}}
$

The F1 score is the **harmonic mean** of precision and recall. It penalizes imbalance: The F1 score is low if either precision *or* recall is low. A model cannot achieve a high F1 score by excelling at one and ignoring the other.

### Question 2

*2 pts.*

Compute the F1 score for both the kNN recommendations and mean baseline, based on the previously computed precision and recall scores. Print the results.

In [ ]:
# your code here

print(f"F1 score kNN: {f1_knn:.2f}")
print(f"F1 score mean: {f1_mean:.2f}")

In [ ]:
# test your solution
tests_m1.real_data_02(f1_knn, f1_mean)

# Precision–Recall Curve

Up to now, we have evaluated precision and recall using a single fixed threshold. But in practice, the threshold is a tunable parameter: changing it can dramatically alter the behaviour of the recommender system.

If we **increase** the threshold, we become more conservative in our recommendations. Only the highest predicted ratings count as recommended. This usually **increases precision** (fewer incorrect recommendations) but **decreases recall** (we miss more potentially good recommendations).

A precision–recall curve helps us understand this trade-off.

A **precision–recall (PR) curve** shows how precision and recall change as we vary the classification threshold. For every possible threshold value, we:

* convert predicted ratings into recommended / not recommended,
* compute precision,
* compute recall,
* and plot the resulting point.

Connecting all these points yields the PR curve.

Ideally, if the algorithm works well, we should be able to find points on the curve where **both precision and recall are relatively high**. This would mean there exists a threshold at which the recommender system performs well on both fronts. In such cases, the PR curve will tend toward the **upper-right corner** of the plot.

For example:

<img src='pr_curve.png' width="350pt">

### Challenge 1

*6 pts.*

Create **precision–recall curves** for both the **kNN algorithm** and the **baseline**.
Vary the recommendation threshold from **1.5 to 4.5** in steps of **0.05**, and compute the **precision** and **recall** at each threshold. Store these values and use them to plot the two PR curves.

Make sure that:

* both curves are clearly labeled,
* the axes are labeled,
* and the visualization is accessible for colorblind readers (e.g., by using distinct line styles in addition to color).

**You need to discuss your solution in person to receive these points.**

In [ ]:
# your code here

### Challenge 2

*4 pts.*

On both PR curves (kNN and baseline), identify and mark the point where the **F1 score** is highest. For both curves:

* mark the corresponding point on the plot,
* and display the **threshold**, **precision**, and **recall** values at which this maximum F1 score occurs.

Make sure these points are clearly distinguishable on the graph.

**You need to discuss your solution in person to receive these points.**

### Challenge 3

*4 pts.*

Create a new precision–recall curve for **only the kNN model**, but this time:

* **fix the threshold at `3.1`**, and
* **vary the value of `k`**.

Determine an appropriate range of `k` values to explore (you should pick no more than about 10 values, since rerunning the kNN algorithm for each `k` is computationally expensive).

For each chosen `k`:

* recompute the predictions,
* convert them into recommendations using the fixed threshold,
* compute precision and recall,
* and plot the resulting PR curve.

**You need to discuss your solution in person to receive these points.**